# Training module

In [1]:
#import library
import csv
import pandas as pd
import pickle 
import numpy as np
import cv2
import mediapipe as mp
import time
import shutil
import os
import sys

from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score 
import joblib

prj_path = r"C:\Users\Chale\Desktop\Project ITM65\Model Framework\Code\Web-applications-for-elderly-physical-exercises\ElderFitmodel"

# Test Camera

In [2]:
# Grabbing the Holistic Model from Mediapipe and
mp_holistic = mp.solutions.holistic

# Initializing the Model
holistic_model = mp_holistic.Holistic(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)
 
# Initializing the drawing utils for drawing the facial landmarks on image
mp_drawing = mp.solutions.drawing_utils

In [18]:
# (0) in VideoCapture is used to connect to your computer's default camera
capture = cv2.VideoCapture(0)
 
# Initializing current time and precious time for calculating the FPS
previousTime = 0
currentTime = 0

start_time = time.time()

while capture.isOpened():
    # capture frame by frame
    ret, frame = capture.read()
    
    elapsed_time = time.time() - start_time
    
    # resizing the frame for better view
    frame = cv2.resize(frame, (860,645))
 
    # Converting the from BGR to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
 
    # Making predictions using holistic model
    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    results = holistic_model.process(image)
    image.flags.writeable = True
 
    # Converting back the RGB image to BGR
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
 
    mp_drawing.draw_landmarks(
        image, 
        results.pose_landmarks, 
        mp_holistic.POSE_CONNECTIONS,            
    )
    # Calculating the FPS
    currentTime = time.time()
    fps = 1 / (currentTime-previousTime)
    previousTime = currentTime
    
    # represents the top left corner of rectangle
    start_point = (200, 10)

    # represents the bottom right corner of rectangle
    end_point = (680, 630)

    # Blue color in BGR
    color = (0,255,255)

    # Line thickness of 2 px
    thickness = 4

    # Using cv2.rectangle() method
    # Draw a rectangle with blue line borders of thickness of 2 px
    cv2.rectangle(image, start_point, end_point, color, thickness)
    
    
    # Displaying FPS on the image
    cv2.putText(image, str(int(fps))+" FPS", (10, 70), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
    
    # Display Timer
    cv2.putText(image, 'TIME'
                , (300,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(image, f"{int(elapsed_time)} Sec"
                , (350,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
 
    # Display the resulting image
    cv2.imshow("Test Camera", image)

    # Enter key 'q' to break the loop
    if cv2.waitKey(5) & 0xFF == ord('q') or cv2.waitKey(5) & int(elapsed_time) == 10:
        break
 
# When all the process is done
# Release the capture and destroy all windows
capture.release()
cv2.destroyAllWindows()

In [19]:
class_name = stretchOutAndStepBack[3]
#class_name = "dance"

# Absolute path of a file
go_src_path = prj_path+"\Dataset\predataset.csv"
go_dst_path = prj_path+"\Dataset\\"+model_name+"\predataset.csv"
data_path = prj_path+"\Dataset\\"+model_name+"\\"+class_name+".csv"

# Copying and rename the file
if not os.path.isfile(data_path):
    shutil.copy(go_src_path, go_dst_path)
    os.rename(go_dst_path, data_path)

In [20]:
start_time = time.time()

# (0) in VideoCapture is used to connect to your computer's default camera
capture = cv2.VideoCapture(0)
 
while capture.isOpened():
    # capture frame by frame
    ret, frame = capture.read()
    
    elapsed_time = time.time() - start_time
 
    # resizing the frame for better view
    frame = cv2.resize(frame, (860,645))
 
    # Converting the from BGR to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
 
    # Making predictions using holistic model
    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    results = holistic_model.process(image)
    image.flags.writeable = True
 
    # Converting back the RGB image to BGR
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    mp_drawing.draw_landmarks(
        image, 
        results.pose_landmarks, 
        mp_holistic.POSE_CONNECTIONS,            
    )
    
    try:
        pose = results.pose_landmarks.landmark
        pose_row = list(np.array([[landmark.x,landmark.y,landmark.z,landmark.visibility] for landmark in pose]).flatten())

        pose_row.insert(0, class_name)
            
        with open(data_path, mode='a', newline='') as f:
            csv_writer = csv.writer(f, delimiter=',')
            csv_writer.writerow(pose_row)
        
        # represents the top left corner of rectangle
        start_point = (200, 10)
        
        # represents the bottom right corner of rectangle
        end_point = (680, 630)
        
        # Blue color in BGR
        color = (0,255,255)
        
        # Line thickness of 2 px
        thickness = 4
        
        # Using cv2.rectangle() method
        # Draw a rectangle with blue line borders of thickness of 2 px
        cv2.rectangle(image, start_point, end_point, color, thickness)

        # Get status box
        cv2.rectangle(image, (0,0), (500, 60), (255, 255, 255), -1)
        
        # Display Timer
        cv2.putText(image, 'TIME'
                    , (300,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, f"{int(elapsed_time)} Sec"
                    , (350,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            
    except:
        cv2.putText(image, str('Out of Frame')
                    , (210,150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
    

    # Display the resulting image
    cv2.imshow("Test Data", image)
 
    # Enter key 'q' to break the loop
    if cv2.waitKey(5) & 0xFF == ord('q') or cv2.waitKey(5) & int(elapsed_time) == 90:
        break
 
# When all the process is done
# Release the capture and destroy all windows
capture.release()
cv2.destroyAllWindows()

# Create datasheet

In [21]:
prj_path = r"C:\Users\Chale\Desktop\Project ITM65\Model Framework\Code\Web-applications-for-elderly-physical-exercises\ElderFitmodel"

In [22]:
model_name = "stretchOutAndStepBack"

#create folder of the model
modelFolerPath = prj_path+"\Dataset/"+model_name
if not os.path.exists(modelFolerPath):
    os.makedirs(modelFolerPath)

In [23]:
waistFeetAndLegRaises = ["wflr0","wflr1","wflr2","wflr3"]

In [24]:
stompingAndBent = ["sab0","sab1","sab2"]

In [25]:
fistAndStride = ["fas0","fas1","fas2","fas3"]

In [26]:
stretchOutAndStepBack  = ["ss0","ss1","ss2","ss3"]

# Data acquisition

# TRAIN MODEL

In [27]:
model_name = "stretchOutAndStepBack"

modelFolerPath = prj_path+"\Dataset/"+model_name

In [28]:
# iterate over files in data directory
list_class = []
list_data = []
directory = modelFolerPath+"/"
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        list_data.append(pd.read_csv(f))
        x = filename.split(".")
        list_class.append(x[0])

In [29]:
sys.stdout.write("[  There is " + str(len(list_class)) + " class for this model  ]\n")
print("==============================================================================")
for i in list_class:
    print(i)

print("==============================================================================")
df_model = pd.concat(list_data,ignore_index=True)
print("df_model is now complete")

[  There is 7 class for this model  ]
sit
ss0
ss1
ss2
ss3
stand
walk
df_model is now complete


In [30]:
df_model = df_model.drop(0)

In [31]:
df_model 

class        x1        y1        z1        v1        x2        y2  \
1       sit  0.369607  0.358517 -0.345011  0.999960  0.378441  0.340451   
2       sit  0.369935  0.355136 -0.369356  0.999963  0.379445  0.338967   
3       sit  0.369443  0.355762 -0.415233  0.999966  0.379896  0.337772   
4       sit  0.370909  0.356149 -0.434472  0.999969  0.381398  0.338802   
5       sit  0.371882  0.359737 -0.423884  0.999971  0.381925  0.341544   
...     ...       ...       ...       ...       ...       ...       ...   
13553  walk  0.375721  0.074755 -0.203680  0.999436  0.369221  0.058373   
13554  walk  0.405675  0.075380 -0.274981  0.999478  0.399794  0.057924   
13555  walk  0.431521  0.075075 -0.294875  0.999518  0.426292  0.057489   
13556  walk  0.442688  0.077119 -0.300460  0.999499  0.438099  0.059013   
13557  walk  0.453071  0.081867 -0.275479  0.999494  0.448264  0.063807   

             z2        v2        x3  ...       z31       v31       x32  \
1     -0.321608  0.999962  0.384194  ... -0.374723  0.733850  0.527517   
2     -0.344952  0.999965  0.385136  ... -0.354610  0.748255  0.516663   
3     -0.391031  0.999968  0.386044  ... -0.357116  0.761563  0.511456   
4     -0.407896  0.999970  0.387799  ... -0.362769  0.773955  0.510629   
5     -0.399117  0.999972  0.388167  ... -0.351143  0.781410  0.508021   
...         ...       ...       ...  ...       ...       ...       ...   
13553 -0.187952  0.998986  0.369507  ...  0.149129  0.623300  0.461684   
13554 -0.257307  0.999063  0.400405  ...  0.189618  0.644691  0.459132   
13555 -0.275408  0.999129  0.426570  ...  0.205530  0.664382  0.453021   
13556 -0.280598  0.999048  0.438478  ...  0.218055  0.683668  0.451181   
13557 -0.257166  0.999002  0.448606  ...  0.183182  0.701462  0.455879   

            y32       z32       v32       x33       y33       z33       v33  
1      1.051075 -0.615228  0.704782  0.258331  1.063047 -0.577261  0.672634  
2      1.048906 -0.581019  0.727163  0.249827  1.061451 -0.563218  0.696407  
3      1.056166 -0.561532  0.744785  0.231345  1.069345 -0.578741  0.716899  
4      1.050850 -0.563531  0.763611  0.238887  1.057677 -0.592985  0.737651  
5      1.048691 -0.560738  0.776551  0.235924  1.058815 -0.562270  0.750991  
...         ...       ...       ...       ...       ...       ...       ...  
13553  0.975862  0.336845  0.726455  0.300782  1.066792  0.009269  0.688695  
13554  0.983548  0.437276  0.744057  0.315667  1.067534  0.040548  0.714595  
13555  0.981769  0.446030  0.755130  0.377171  1.050129  0.056272  0.735070  
13556  0.980700  0.494980  0.751925  0.431793  1.000979  0.062790  0.751414  
13557  0.967596  0.429593  0.740247  0.466825  0.990490  0.046469  0.763903  

[13557 rows x 133 columns]

In [32]:
x = df_model .drop('class', axis=1) 
y = df_model ['class']

In [33]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=999)

In [34]:
pipelines = {
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier(), verbose = 1)
}

In [35]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(x_train, y_train)
    fit_models[algo] = model

[Pipeline] .... (step 1 of 2) Processing standardscaler, total=   0.0s
[Pipeline]  (step 2 of 2) Processing randomforestclassifier, total=   4.8s


In [36]:
for algo, model in fit_models.items():
    yhat = model.predict(x_test)
    print(algo, accuracy_score(y_test, yhat))

rf 0.9977876106194691


In [37]:
model_dst=prj_path+"\Model/"
save_dst=model_dst+'stretchOutAndStepBack.pkl'
pickle.dump(fit_models['rf'],open(save_dst, 'wb'))